In [230]:
# pip install torch

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.2/890.2 MB 1.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 KB 103.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 92.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.0 MB/s eta 0:00:0000:0100:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from sklearn.metrics import f1_score, fbeta_score, precision_score, recall_score, accuracy_score
# from build_model import initialize_model, load_split_data, build_loader, evaluate, train_model
# from build_model import plot_confusion
import os
from sklearn.metrics import confusion_matrix


In [2]:
MODEL_DIR = '/home/ec2-user/SageMaker/teledermatologyAI_capstone/model/gridsearch'


In [3]:
cd $MODEL_DIR

/home/ec2-user/SageMaker/teledermatologyAI_capstone/model/gridsearch


In [ ]:
[model, 'file', 'tuned_model', 'transform', 'lr', 'pretrained_model',
       'optimizer', 'epochs', 'num_classes', 'batch_size', 'workers',
       'train_time', 'data_split', 'label_set', 'accur', 'F1', 'F0.5', 'F2',
       'benign_accur', 'noncancerous_accur', 'malignant_accur',
       'infection_accur', 'unclassified_accur'],

In [25]:
r = pd.read_csv('results_gs2.csv')
# type(r['model'][0])
r_dict = dict()

for i in r.columns:
    print(i, type(r[i][0]))
    r_dict[i] = type(r[i][0])

r_dict

model <class 'numpy.int64'>
file <class 'str'>
tuned_model <class 'str'>
transform <class 'numpy.int64'>
lr <class 'numpy.float64'>
pretrained_model <class 'str'>
optimizer <class 'str'>
epochs <class 'numpy.int64'>
num_classes <class 'numpy.int64'>
batch_size <class 'numpy.int64'>
workers <class 'numpy.int64'>
train_time <class 'str'>
data_split <class 'str'>
label_set <class 'str'>
accur <class 'numpy.float64'>
F1 <class 'numpy.float64'>
F0.5 <class 'numpy.float64'>
F2 <class 'numpy.float64'>
benign_accur <class 'numpy.float64'>
noncancerous_accur <class 'numpy.float64'>
malignant_accur <class 'numpy.float64'>
infection_accur <class 'numpy.float64'>
unclassified_accur <class 'numpy.float64'>


{'model': numpy.int64,
 'file': str,
 'tuned_model': str,
 'transform': numpy.int64,
 'lr': numpy.float64,
 'pretrained_model': str,
 'optimizer': str,
 'epochs': numpy.int64,
 'num_classes': numpy.int64,
 'batch_size': numpy.int64,
 'workers': numpy.int64,
 'train_time': str,
 'data_split': str,
 'label_set': str,
 'accur': numpy.float64,
 'F1': numpy.float64,
 'F0.5': numpy.float64,
 'F2': numpy.float64,
 'benign_accur': numpy.float64,
 'noncancerous_accur': numpy.float64,
 'malignant_accur': numpy.float64,
 'infection_accur': numpy.float64,
 'unclassified_accur': numpy.float64}

In [49]:
r

,model,file,tuned_model,transform,lr,pretrained_model,optimizer,epochs,num_classes,batch_size,...,label_set,accur,F1,F0.5,F2,benign_accur,noncancerous_accur,malignant_accur,infection_accur,unclassified_accur
0,0,resnet_split3_3e_GS0_preds.pkl,resnet_split3_3e_GS0,0,0.001,resnet,SGD,3,5,64,...,label_0,0.587778,0.589663,0.598624,0.586436,0.662362,0.720974,0.5369,0.453704,0.566421


In [48]:
# Create empty df
col_dict = {
#              'model': pd.Series(dtype = 'int'),
#              'file': pd.Series(dtype = 'str'),
             'tuned_model': pd.Series(dtype = 'str'),
             'transform': pd.Series(dtype = 'int'),
             'lr': pd.Series(dtype = 'float'),
             'pretrained_model': pd.Series(dtype = 'str'),
             'optimizer': pd.Series(dtype = 'str'),
             'epochs': pd.Series(dtype = 'int'),
             'num_classes': pd.Series(dtype = 'int'),
             'batch_size': pd.Series(dtype = 'int'),
             'workers': pd.Series(dtype = 'int'),
             'train_time': pd.Series(dtype = 'str'),
             'data_split': pd.Series(dtype = 'str'),
             'label_set': pd.Series(dtype = 'str'),
             'accur': pd.Series(dtype = 'float'),
             'F1': pd.Series(dtype = 'float'),
             'F0.5': pd.Series(dtype = 'float'),
             'F2': pd.Series(dtype = 'float'),
             'benign_accur': pd.Series(dtype = 'float'),
             'noncancerous_accur': pd.Series(dtype = 'float'),
             'malignant_accur': pd.Series(dtype = 'float'),
             'infection_accur': pd.Series(dtype = 'float'),
             'unclassified_accur': pd.Series(dtype = 'float')}
results = pd.DataFrame(col_dict)

In [45]:
# for col in r_dict.keys():
#     print(col, r_dict[col])

# results = pd.DataFrame({})


In [52]:
pd.concat([results, r])

,model,file,tuned_model,transform,lr,pretrained_model,optimizer,epochs,num_classes,batch_size,...,label_set,accur,F1,F0.5,F2,benign_accur,noncancerous_accur,malignant_accur,infection_accur,unclassified_accur
0,0,resnet_split3_3e_GS0_preds.pkl,resnet_split3_3e_GS0,0,0.001,resnet,SGD,3,5,64,...,label_0,0.587778,0.589663,0.598624,0.586436,0.662362,0.720974,0.5369,0.453704,0.566421


In [4]:
# Load files in directory, filter to pred files
files = pd.Series(os.listdir('./'), name = "file")
files = pd.DataFrame(files[files.str.contains('pkl')])

# extract model number and name, convert model num to int
files['model_number'] = files.file.apply(lambda st: st[st.find("GS")+2:st.find("_pred")])
files['tuned_model'] = files.file.apply(lambda st: st[:st.find("_pred")])
files['model_number'] = files['model_number'].astype('int')
files.set_index('model_number', inplace = True)

# read in prior partial results
gs = pd.read_csv('gridsearch1.csv')

In [5]:
# Create results df by combining hyperparams and model name
results = pd.concat([files, gs], axis = 1).set_index('model')

In [9]:
results = results[:1]

In [32]:
# new columns
results['pretrained_model'] = 'resnet'
results['optimizer'] = 'SGD'
results['epochs'] = 3
results['num_classes'] = 5
results['batch_size'] = 64
results['workers'] = 24
results['train_time'] = 'unk'
results['data_split'] = 'split_3'
results['label_set'] = 'label_0'
results['accur'] = None
results['F1'] = None
results['F0.5'] = None
results['F2'] = None
results['benign_accur'] = None
results['noncancerous_accur'] = None
results['malignant_accur'] = None
results['infection_accur'] = None
results['unclassified_accur'] = None



In [11]:
# i = 1
for index, row in results.iterrows():
#     row.iloc[index]['accur'] = i
    
    d = pd.read_pickle(row['file'])
    
    # Scores
    acc = accuracy_score(d.lab, d.pred)
    f1 = f1_score(d.lab, d.pred, average = 'macro')
    f2 = fbeta_score(d.lab, d.pred, average = 'macro', beta = 2)
    f5 =  fbeta_score(d.lab, d.pred, average = 'macro', beta = .5)
    prec = precision_score(d.lab, d.pred, average = 'macro')
    rec = recall_score(d.lab, d.pred, average = 'macro')
    
    # confusion matrix 
    c_matrix = confusion_matrix(d.lab, d.pred, normalize = 'true')

    results.at[index, 'accur'] = acc
    results.at[index, 'F1'] = f1
    results.at[index, 'F0.5'] = f5
    results.at[index, 'F2'] = f2
    results.at[index, 'benign_accur'] = c_matrix[0,0]
    results.at[index, 'noncancerous_accur'] = c_matrix[1,1]
    results.at[index, 'malignant_accur'] = c_matrix[2,2]
    results.at[index, 'infection_accur'] = c_matrix[3,3]
    results.at[index, 'unclassified_accur'] = c_matrix[4,4]
#     i+=1

In [12]:
results.to_csv('./results_gs2.csv')

In [33]:
a = model_scores(d.lab, d.pred)

In [28]:
def model_scores(true_labs, preds):
    acc = accuracy_score(d.lab, d.pred)
    f1 = f1_score(d.lab, d.pred, average = 'macro')
    f2 = fbeta_score(d.lab, d.pred, average = 'macro', beta = 2)
    f5 =  fbeta_score(d.lab, d.pred, average = 'macro', beta = .5)
    prec = precision_score(d.lab, d.pred, average = 'macro')
    rec = recall_score(d.lab, d.pred, average = 'macro')
    
    # confusion matrix & Diags
    c_matrix = confusion_matrix(d.lab, d.pred, normalize = 'true')
    d_0 = c_matrix[0,0]
    d_1 = c_matrix[1,1]
    d_2 = c_matrix[2,2]
    d_3 = c_matrix[3,3]
    d_4 = c_matrix[4,4]
    
    return acc, f1, f2, f5, prec, rec, d_0, d_1, d_2, d_3, d_4

In [35]:
a[0]

0.7022222222222222

In [13]:
results.columns

Index(['file', 'tuned_model', 'transform', 'lr', 'pretrained_model',
       'optimizer', 'epochs', 'num_classes', 'batch_size', 'workers',
       'train_time', 'data_split', 'label_set', 'accur', 'F1', 'F0.5', 'F2',
       'benign_accur', 'noncancerous_accur', 'malignant_accur',
       'infection_accur', 'unclassified_accur'],
      dtype='object')